# 2-D Raman Mapping Notebook — Quick Intro

This notebook does four things:

1. **Load data**  
   *Reads a baseline-subtracted `.txt` file (metadata header + spectra) and reshapes it into a 21 × 21 × *N* array.*

2. **Show an interactive Raman map**  
   *A slider lets you pick any Raman shift; hovering shows the local intensity; double-clicking adds that pixel’s full spectrum to a side plot. The helper that drives this GUI is `plot_2d_raman` in `function_lib.py`* :contentReference[oaicite:0]{index=0}

3. **(Optional) Fit peaks everywhere**  
   *Cells later in the notebook call fitting routines (Lorentzian, double-Lorentzian, Voigt) and build parameter matrices through `parameter_matrix`* :contentReference[oaicite:1]{index=1}

4. **Save or plot results**  
   *Buttons and helper functions let you export selected coordinates, peak maps, or example fits.*

In [1]:
# ======================================================================
# Cell 1 – Load libraries and raw Raman data
# ----------------------------------------------------------------------
# 1. Import all required third-party libraries.
# 2. Define the file location (folder + filename) holding the
#    baseline-subtracted Raman export produced by Renishaw Wire.
# 3. Open the text file, skip the 55-line header and read the numerical
#    data into a list of floats.
# 4. Parse that list into:
#       • wave_numbers      – 1-D array of Raman shifts (cm-1)
#       • coordinates       – N×2 array of (y, x) positions (µm)
#       • raman_amplitudes  – N×N_w intensity matrix
# 5. Reshape the 1-D vectors into 21×21 spatial grids that are convenient
#    for heat-maps and spectral exploration in later cells.
# ======================================================================

# --- 1. Imports -------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec
from scipy.optimize import curve_fit
from scipy.special import wofz
import function_lib as fn          # local helper routines
from scipy.optimize import curve_fit

# --- 2. File location -------------------------------------------------
folder_path = (
    r"C:\Users\ashfa\OneDrive - The Pennsylvania State University\Shao Group files"
)
file_name = (
    r"\G-MoS2_532 nm_2024_11_14_12_25_34_600 gr_mm_x100_1_1 s_100_2_5%_01 Baseline subtracted.txt"
)
file_path = folder_path + file_name

# --- 3. Read the text file -------------------------------------------
data_array = []
with open(file_path, "r") as file:
    # Skip the 55-line metadata header produced by the spectrometer
    for _ in range(55):
        next(file)
    # Each subsequent line:  y  x  I(shift₁) I(shift₂) … I(shift_N_w)
    for line in file:
        data_array.append(list(map(float, line.strip().split())))

# --- 4. Split raw list into logical pieces ---------------------------
wave_numbers = np.array(data_array[0])           # (N_w,)
N_w = wave_numbers.size

coordinates = np.array([row[:2] for row in data_array[1:]])   # (N_pts, 2)
raman_amplitudes = np.array([row[2:] for row in data_array[1:]])  # (N_pts, N_w)

# --- 5. Reshape to 21×21 spatial grids -------------------------------
data_grid = raman_amplitudes.reshape(21, 21, N_w)  # intensity cube
x_grid = coordinates[:, 1].reshape(21, 21)         # x-positions
y_grid = coordinates[:, 0].reshape(21, 21)         # y-positions


c:\Users\ashfa\OneDrive - The Pennsylvania State University\Shao Group files\function_lib.py:15: SyntaxWarning: invalid escape sequence '\m'
  """
c:\Users\ashfa\OneDrive - The Pennsylvania State University\Shao Group files\function_lib.py:150: SyntaxWarning: invalid escape sequence '\m'
  """


In [5]:
# =====================================================================
# Cell 2 – Build an interactive Raman-map viewer
# ---------------------------------------------------------------------
# *Prerequisites:* variables `wave_numbers`, `coordinates`, `raman_amplitudes`,
#                 `data_grid` were created in Cell 1.
#
# What this cell creates
# ----------------------
# 1. A figure with two main panels:
#       • ax_map     : false-colour Raman-intensity map for a chosen shift
#       • ax_spectra : overlaid spectra of every pixel the user double-clicks
# 2. A slider that lets the user scroll through Raman shifts (frame index).
# 3. A “Clear” button to remove markers and reset the spectra panel.
# 4. Hover read-outs: live display of intensity under the cursor.
#
# User actions
# ------------
# • Drag slider → updates the 2-D map to that Raman shift.
# • Hover over map → shows intensity at cursor (top-left read-out).
# • Double-click a pixel → places an “×” marker and plots its full spectrum.
# • Press *Clear* → removes all markers/spectra and resets the GUI.
# =====================================================================

# ---------------------------------------------------------------------
# 0. Quick helpers from previous cell
# ---------------------------------------------------------------------
N_w = len(wave_numbers)                           # total number of spectral frames

# Reshape linear coordinate arrays into 21×21 grids for pcolormesh
x_grid = coordinates[:, 1].reshape(21, 21)
y_grid = coordinates[:, 0].reshape(21, 21)

# ---------------------------------------------------------------------
# 1. Create the figure layout: 2×2 GridSpec
# ---------------------------------------------------------------------
fig = plt.figure(figsize=(15, 8))
gs  = gridspec.GridSpec(
    nrows=2, ncols=2,
    height_ratios=[2, 1],    # top row twice as tall as bottom
    width_ratios=[2, 1]      # left panel twice as wide as right
)

ax_map     = fig.add_subplot(gs[:, 0])   # spans both rows
ax_spectra = fig.add_subplot(gs[0, 1])   # top-right
fig.subplots_adjust(left=0.05, right=0.95, wspace=0.15)

# ---------------------------------------------------------------------
# 2. Initial Raman-map (frame 0) + colour bar
# ---------------------------------------------------------------------
scatter = ax_map.pcolormesh(
    x_grid, y_grid, data_grid[:, :, 0],
    cmap='viridis', vmax=250
)
cbar = fig.colorbar(scatter, ax=ax_map, pad=0.02, aspect=20)
cbar.set_label('Intensity (counts)', fontsize=14)
cbar.ax.tick_params(labelsize=12)

ax_map.set_xlabel('X (µm)', fontsize=16)
ax_map.set_ylabel('Y (µm)', fontsize=16)
ax_map.set_title('Raman Map', fontsize=18, weight='bold')
ax_map.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
ax_map.invert_yaxis()                           # origin-upper ≙ microscope view

# ---------------------------------------------------------------------
# 3. Slider to scroll through Raman shifts
# ---------------------------------------------------------------------
ax_slider = plt.axes([0.25, 0.01, 0.5, 0.03], facecolor='lightgrey')
slider = Slider(ax_slider, label='', valmin=0, valmax=N_w-1,
                valinit=0, valstep=1)
slider.valtext.set_visible(False)               # hide unnecessary read-out

# ---------------------------------------------------------------------
# 4. Read-out text boxes (current shift & intensity at cursor)
# ---------------------------------------------------------------------
ax_text  = plt.axes([0.45, 0.92, 0.1, 0.03], frame_on=False)
ax_text.axis('off')
text = ax_text.text(
    0.5, 0.5,
    f'Wavenumber: {wave_numbers[0]:.2f} cm$^{{-1}}$',
    ha='center', va='center', fontsize=12
)

ax_text2 = plt.axes([0.12, 0.92, 0.1, 0.03], frame_on=False)
ax_text2.axis('off')
text2 = ax_text2.text(
    0.5, 0.5,
    f'Intensity: {data_grid[0, 0, 0]:.2f}',
    ha='center', va='center', fontsize=12
)

# ---------------------------------------------------------------------
# 5. “Clear” button to reset selections
# ---------------------------------------------------------------------
ax_button   = plt.axes([0.88, 0.02, 0.08, 0.04])
reset_button = Button(ax_button, 'Clear', color='lightgrey', hovercolor='0.975')

# ---------------------------------------------------------------------
# 6. State containers for clicked points
# ---------------------------------------------------------------------
clicked_indices = []            # linear indices of already-picked pixels
map_markers     = []            # Matplotlib Line2D handles
color_list      = list(mcolors.TABLEAU_COLORS)   # distinct colours for spectra

# ---------------------------------------------------------------------
# 7. Callback: slider → update heat-map & read-out
# ---------------------------------------------------------------------
def update(val):
    wn = int(slider.val)                                # current frame index
    scatter.set_array(data_grid[:, :, wn].ravel())      # refresh colours
    cbar.update_normal(scatter)                         # update colour scale
    text.set_text(f'Wavenumber: {wave_numbers[wn]:.2f} cm$^{{-1}}$')
    fig.canvas.draw_idle()

# ---------------------------------------------------------------------
# 8. Callback: mouse hover → show intensity under cursor
# ---------------------------------------------------------------------
def on_hover(event):
    if event.inaxes != ax_map or event.xdata is None:   # ignore outside map
        return
    x_hover, y_hover = event.xdata, event.ydata
    # Euclidean distance to every pixel centre; pick nearest
    distances    = np.hypot(coordinates[:, 1] - x_hover,
                             coordinates[:, 0] - y_hover)
    closest_idx  = np.argmin(distances)
    wn           = int(slider.val)
    intensity    = raman_amplitudes[closest_idx, wn]
    text2.set_text(f'Intensity: {intensity:.2f}')
    fig.canvas.draw_idle()

# ---------------------------------------------------------------------
# 9. Callback: double-click → mark pixel & plot its spectrum
# ---------------------------------------------------------------------
def on_click(event):
    if event.inaxes != ax_map or not event.dblclick:
        return
    x_click, y_click = event.xdata, event.ydata
    distances   = np.hypot(coordinates[:, 1] - x_click,
                            coordinates[:, 0] - y_click)
    closest_idx = int(np.argmin(distances))

    # Prevent duplicates or overflow
    if closest_idx in clicked_indices:
        print(f"Point at index {closest_idx} already selected.")
        return
    if len(clicked_indices) >= len(color_list):
        print("Reached colour limit; no more points can be added.")
        return

    # Store selection
    clicked_indices.append(closest_idx)
    colour = color_list[len(clicked_indices) - 1]

    # ---- Map marker
    marker, = ax_map.plot(
        coordinates[closest_idx, 1],
        coordinates[closest_idx, 0],
        marker='x', color=colour, markersize=10, markeredgewidth=2
    )
    map_markers.append(marker)

    # ---- Plot full spectrum
    spectrum = raman_amplitudes[closest_idx, :]
    label    = f'X={coordinates[closest_idx,1]:.2f}, Y={coordinates[closest_idx,0]:.2f}'
    ax_spectra.plot(wave_numbers, spectrum, color=colour, label=label)
    # (legend omitted to avoid clutter; uncomment next line if desired)
    # ax_spectra.legend(loc='upper right', fontsize=10)

    fig.canvas.draw_idle()

# ---------------------------------------------------------------------
# 10. Callback: “Clear” button → reset everything
# ---------------------------------------------------------------------
def reset(event):
    global clicked_indices, map_markers
    clicked_indices.clear()

    # Remove markers from map
    for marker in map_markers:
        marker.remove()
    map_markers.clear()

    # Clear spectra panel & restore labels
    ax_spectra.cla()
    ax_spectra.set_xlabel('Wavenumber (cm$^{-1}$)', fontsize=14)
    ax_spectra.set_ylabel('Intensity (counts)', fontsize=14)
    ax_spectra.set_title('Raman Spectra at Selected Points', fontsize=16)
    ax_spectra.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)

    fig.canvas.draw_idle()

# ---------------------------------------------------------------------
# 11. Register callbacks with the GUI elements
# ---------------------------------------------------------------------
reset_button.on_clicked(reset)
slider.on_changed(update)
fig.canvas.mpl_connect('motion_notify_event',  on_hover)
fig.canvas.mpl_connect('button_press_event',   on_click)

# ---------------------------------------------------------------------
# 12. Final cosmetics for spectra panel, then launch the GUI
# ---------------------------------------------------------------------
ax_spectra.set_xlabel('Wavenumber (cm$^{-1}$)', fontsize=14)
ax_spectra.set_ylabel('Intensity (counts)',    fontsize=14)
ax_spectra.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)

plt.show()      # blocking call – user interacts until window is closed


In [ ]:
# =====================================================================
# Cell 3 – Quick point-selector on a single 2-D Raman frame
# ---------------------------------------------------------------------
# 1. Show a simple 2-panel GUI:
#       • Left: 2-D intensity map for a chosen Raman-shift frame
#       • Bottom: slider to move through spectral frames
# 2. Let the user click on pixels (single click) to mark points of
#    interest – each click drops a red “×” and stores the (x, y) coords.
# 3. A *Save* button writes all picked coordinates to
#       "selected_points.txt"  (X, Y per line, micrometres).
# ---------------------------------------------------------------------
# *Prerequisites:* x_grid, y_grid, data_grid, wave_numbers, coordinates,
#                  N_w  – created in previous cells.
# =====================================================================

# ---- 1. Figure & initial 2-D Raman map ------------------------------
fig, ax_map = plt.subplots(figsize=(8, 6))
plt.subplots_adjust(bottom=0.2)                       # leave room for slider

scatter = ax_map.pcolormesh(
    x_grid, y_grid, data_grid[:, :, 0],
    cmap='viridis', vmax=250
)

cbar = fig.colorbar(scatter, ax=ax_map, pad=0.02, aspect=20)
cbar.set_label('Intensity (counts)', fontsize=14)

ax_map.set_xlabel('X (µm)', fontsize=14)
ax_map.set_ylabel('Y (µm)', fontsize=14)
ax_map.set_title('Raman Map', fontsize=16, weight='bold')
ax_map.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
ax_map.invert_yaxis()                                # microscope view

# ---- 2. Slider to choose Raman-shift frame ---------------------------
ax_slider = plt.axes([0.25, 0.05, 0.5, 0.03], facecolor='lightgrey')
slider = Slider(ax_slider, 'Frame', 0, N_w-1, valinit=0, valstep=1)

def update(val):
    """Callback: refresh map when slider moves."""
    frame = int(slider.val)
    scatter.set_array(data_grid[:, :, frame].ravel())
    cbar.update_normal(scatter)
    fig.canvas.draw_idle()

slider.on_changed(update)

# ---- 3. Point picking on single click --------------------------------
selected_points = []          # list of (x, y) in µm

def on_click(event):
    """Callback: add a red ‘×’ marker and store the coordinate."""
    if event.inaxes != ax_map or event.xdata is None:
        return
    x_click, y_click = event.xdata, event.ydata
    # find nearest pixel centre
    dist = np.hypot(coordinates[:, 1] - x_click,
                    coordinates[:, 0] - y_click)
    idx = int(np.argmin(dist))
    xy  = coordinates[idx, 1], coordinates[idx, 0]   # (x, y)
    selected_points.append(xy)
    ax_map.plot(*xy, marker='x', color='red', markersize=10)
    fig.canvas.draw_idle()

fig.canvas.mpl_connect('button_press_event', on_click)

# ---- 4. Save button --------------------------------------------------
def save_points(event):
    """Write selected_points → text file (two cols: X, Y in µm)."""
    with open("selected_points.txt", "w") as f:#change the name
        for x, y in selected_points:
            f.write(f"{x:.2f}, {y:.2f}\n")
    print("Selected points saved to 'selected_points.txt'.")

ax_button   = plt.axes([0.8, 0.05, 0.1, 0.04])
save_button = Button(ax_button, 'Save', color='lightgrey', hovercolor='0.975')
save_button.on_clicked(save_points)

plt.show()          # interactive window


In [11]:
# =====================================================================
# Cell 5 – Helper routines for Lorentzian peak fitting
# ---------------------------------------------------------------------
# Contents:
#   • lorentzian(x, A, x0, gamma, B)
#       Classic 4-parameter Lorentzian line-shape.
#   • wn_indices(neighbourhood, wave_numbers)
#       Convert a (min, max) cm-¹ window to the corresponding index
#       range in the 1-D `wave_numbers` axis.
#   • parameter_matrix(neighbourhood, data_grid, wave_numbers)
#       Slice every pixel’s spectrum to the chosen neighbourhood,
#       fit a Lorentzian, and return a 3-D cube of best-fit parameters:
#           parameters[i, j] = [A, x0, gamma, B]
# ---------------------------------------------------------------------
# These utilities are called from plotting cells to map peak position,
# FWHM or amplitude across the entire 21×21 scan.
# =====================================================================

import numpy as np
from scipy.optimize import curve_fit

# ---------------------------------------------------------------------
# 1. Lorentzian model --------------------------------------------------
def lorentzian(x, A, x0, gamma, B):
    """
    Lorentzian peak function.

    Parameters
    ----------
    x : ndarray
        Raman-shift axis (cm⁻¹).
    A : float
        Peak amplitude.
    x0 : float
        Centre (peak position, cm⁻¹).
    gamma : float
        Half-width at half-maximum (HWHM, cm⁻¹).
    B : float
        Constant baseline offset.

    Returns
    -------
    ndarray
        Lorentzian evaluated at `x`.
    """
    return A / (1 + ((x - x0) / gamma) ** 2) + B


# ---------------------------------------------------------------------
# 2. Helper: window → index range -------------------------------------
def wn_indices(neighbourhood, wave_numbers):
    """
    Convert a (min, max) wavenumber window into start & end indices
    on the `wave_numbers` grid.

    Parameters
    ----------
    neighbourhood : tuple | list (len==2)
        Lower and upper bounds of the desired window (cm⁻¹).
    wave_numbers : ndarray
        Full Raman-shift axis for the experiment.

    Returns
    -------
    list[int, int]
        [start_idx, end_idx] bracketing the window.
    """
    start = np.argmin(np.abs(wave_numbers - neighbourhood[0]))
    end   = np.argmin(np.abs(wave_numbers - neighbourhood[1]))
    return [start, end]


# ---------------------------------------------------------------------
# 3. Batch-fit Lorentzians over the 2-D scan ---------------------------
def parameter_matrix(neighbourhood, data_grid, wave_numbers):
    """
    Fit a Lorentzian to every pixel within the chosen wavenumber window.

    Parameters
    ----------
    neighbourhood : tuple | list (len==2)
        (min_cm⁻¹, max_cm⁻¹) defining the fit range.
    data_grid : ndarray, shape (Ny, Nx, N_w)
        3-D cube of Raman intensities.
    wave_numbers : ndarray, shape (N_w,)
        Raman-shift axis.

    Returns
    -------
    ndarray, shape (Ny, Nx, 4)
        Best-fit parameters [A, x0, gamma, B] for each pixel.
    """
    # --- Slice spectra to the neighbourhood --------------------------
    idx_start, idx_end = wn_indices(neighbourhood, wave_numbers)
    spectra_slice = data_grid[:, :, idx_start:idx_end]
    wn_slice      = wave_numbers[idx_start:idx_end]

    # --- Prepare output container ------------------------------------
    Ny, Nx, _ = spectra_slice.shape
    params = np.zeros((Ny, Nx, 4))

    # --- Initial guess: [A, x0, gamma, B] ----------------------------
    guess = [10, np.mean(neighbourhood), 50, 10]

    # --- Fit each pixel spectrum -------------------------------------
    for i in range(Ny):
        for j in range(Nx):
            popt, _ = curve_fit(lorentzian, wn_slice, spectra_slice[i, j, :],
                                p0=guess, maxfev=5000)
            params[i, j] = popt

    return params


# ---------------------------------------------------------------------
# 4. Extract G- and 2D-peak positions at user-selected points ----------
def get_peak_vals(
    omega_g_0: float,
    omega_2d_0: float,
    wg_neighbourhood: list | tuple,
    w2d_neighbourhood: list | tuple,
    data_grid: np.ndarray,
    wave_numbers: np.ndarray,
):
    """
    Return G- and 2D-band peak positions for every point listed in
    `selected_points.txt`.  Only pixels whose fitted G-peak is within
    ±60 cm⁻¹ of `omega_g_0` are kept.

    Parameters
    ----------
    omega_g_0 : float
        Reference G-band position (cm⁻¹).
    omega_2d_0 : float
        Reference 2D-band position (cm⁻¹).  (Used only for context.)
    wg_neighbourhood : (low, high)
        Fit window for the G-band.
    w2d_neighbourhood : (low, high)
        Fit window for the 2D-band.
    data_grid : ndarray, shape (Ny, Nx, N_w)
        Full Raman-intensity cube.
    wave_numbers : ndarray, shape (N_w,)
        Raman-shift axis.

    Returns
    -------
    g_peaks : list[float]
        Fitted G-band positions at the selected pixels.
    td_peaks : list[float]
        Fitted 2D-band positions at the same pixels.
    """
    # --- Fit G and 2D bands over the entire scan ---------------------
    omega_g  = parameter_matrix(wg_neighbourhood,  data_grid, wave_numbers)[:, :, 1]
    omega_2d = parameter_matrix(w2d_neighbourhood, data_grid, wave_numbers)[:, :, 1]

    # --- Load user-selected (x, y) coordinates -----------------------
    points_list = np.loadtxt("selected_points.txt", delimiter=",")  # (N_pts, 2)

    # Convert (x, y) in µm → grid indices (n, m)
    grid_indices = []
    for x_pt, y_pt in points_list:
        dist = (x_grid - x_pt) ** 2 + (y_grid - y_pt) ** 2
        grid_indices.append(np.unravel_index(dist.argmin(), x_grid.shape))

    # --- Collect peak positions within a ±60 cm⁻¹ window ------------
    g_peaks, td_peaks = [], []
    for n, m in grid_indices:
        if abs(omega_g[n, m] - omega_g_0) < 60:
            g_peaks.append( omega_g[n, m] )
            td_peaks.append( omega_2d[n, m] )

    return g_peaks, td_peaks
# ---------------------------------------------------------------------


In [10]:
# =====================================================================
# Cell X – Spatial map of the graphene **G-peak position**
# ---------------------------------------------------------------------
# Workflow
#   1. Fit a Lorentzian to the G-band (1400–1700 cm⁻¹) at every pixel
#      using `parameter_matrix` (see previous cell / function_lib.py).
#   2. Extract the fitted peak centre (index 1) → 2-D array `peaks`.
#   3. Show a false-colour map of `peaks` with a hover read-out that
#      displays the exact peak position under the cursor.
# =====================================================================

# --- 1. Batch-fit the G-band -----------------------------------------
neighbourhood   = [1400, 1700]                             # fit window (cm⁻¹)
fit_parameters  = parameter_matrix(neighbourhood,          # → (21, 21, 4)
                                   data_grid,
                                   wave_numbers)

peaks = fit_parameters[:, :, 1]                            # centre frequency

# --- 2. Plot the 2-D peak-position map -------------------------------
fig, ax_peak = plt.subplots(figsize=(12, 10))

scatter_peak = ax_peak.pcolormesh(
    x_grid, y_grid, peaks,
    cmap='coolwarm'
)

# Colour-bar (wavenumber scale)
cbar = fig.colorbar(scatter_peak, ax=ax_peak, pad=0.02, aspect=20)
cbar.set_label('Wavenumber (cm$^{-1}$)', fontsize=14)
cbar.ax.tick_params(labelsize=12)

# Axis labels & title
ax_peak.set_xlabel('X (µm)', fontsize=16)
ax_peak.set_ylabel('Y (µm)', fontsize=16)
ax_peak.set_title('G-Peak Position Map', fontsize=18, weight='bold')
ax_peak.invert_yaxis()

# --- 3. Hover read-out box -------------------------------------------
ax_text2 = plt.axes([0.066, 0.955, 0.1, 0.03], frame_on=False)
ax_text2.axis('off')
text2 = ax_text2.text(
    0.5, 0.5,
    f'Peak: {peaks[0, 0]:.2f}',
    ha='center', va='center', fontsize=12
)

def on_hover(event):
    """Display the peak value at the cursor position."""
    if event.inaxes != ax_peak or event.xdata is None:
        return
    x_hover, y_hover = event.xdata, event.ydata
    # Euclidean distance to pixel centres → nearest pixel indices
    distances = (x_grid - x_hover) ** 2 + (y_grid - y_hover) ** 2
    idx       = np.unravel_index(distances.argmin(), distances.shape)
    peak_val  = peaks[idx]
    text2.set_text(f'Peak: {peak_val:.2f}')
    fig.canvas.draw_idle()

fig.canvas.mpl_connect('motion_notify_event', on_hover)

plt.tight_layout(rect=[0, 0.04, 1, 1])
plt.show()

# (If running in JupyterLab and you prefer the Qt backend, uncomment:)
# %matplotlib qt


C:\Users\ashfa\AppData\Local\Temp\ipykernel_57632\3507220152.py:62: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0.04, 1, 1])


In [18]:
#peak fit example
neighbourhood = [1550,1700]
ind_neigh = wn_indices(neighbourhood, wave_numbers)
A, x0, gamma, B = parameter_matrix(neighbourhood, data_grid, wave_numbers)[13,13]
x_data = wave_numbers[ind_neigh[0]:ind_neigh[1]]
y_fit = lorentzian(x_data, A, x0, gamma, B)
y_data = data_grid[13,13,ind_neigh[0]:ind_neigh[1]]

plt.figure()
plt.plot(x_data, y_data, c = 'C0',label = 'data')
plt.plot(x_data, y_fit, c='red', label = 'fit')
plt.legend()